<a href="https://colab.research.google.com/github/alessandrobruni/TensorflowDiabeticRetinopathy/blob/main/CNN_Diabetic_Retinopathy_loadfiles_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download the images from Kaggle

Kaggle dataset have has been divide into
* retina_severe_CNN.zip  1200 images (folder 0 and 4)
* retina_severe1500_CNN.zip  1200 images (folder 0 and 4)
* retina_severe1800_CNN.zip  1800 images (folder 0 and 4)
* retina_severe3k_CNN.zip  1200 images (folder 0 , 3 and 4)



In [ ]:
#setting up kaggle connection to download the images datasete
# first: install kaggle
!pip install -q kaggle


In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"alebruni","key":"d16ab9c57feefbfbb531c1700a353dc0"}'}

In [ ]:
#make sure kaggle.json file is present
!ls -lha kaggle.json

-rw-r--r-- 1 root root 64 Nov 28 11:22 kaggle.json


In [ ]:
#kaggle API client expects the file to be in ~/.kaggle
#so move it there
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
#we need to set permissions
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
#check your directory before downloading the datasets
!pwd
#download the required dataset from kaggle
!kaggle datasets download -d amanneo/diabetic-retinopathy-resized-arranged

/content
100% 1.20G/1.21G [00:13<00:00, 131MB/s]
100% 1.21G/1.21G [00:13<00:00, 98.7MB/s]


In [ ]:
import zipfile

# Unzip the dataset
local_zip = './diabetic-retinopathy-resized-arranged.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./retina')
zip_ref.close()

# Stucture of the folders
The folder represents specific class labels.
* 0 - No DR
* 1 - Mild
* 2 - Moderate
* 3 - Severe
* 4 - Proliferative DR

Now let's see how many image we have for each classes

In [ ]:
import os

source_path = './retina'

s_path_0_NODR = os.path.join(source_path,'0')
s_path_1_Mild = os.path.join(source_path,'1')
s_path_2_Moderate = os.path.join(source_path,'2')
s_path_3_Severe = os.path.join(source_path,'3')
s_path_4_Proliferative = os.path.join(source_path,'4')

#os..listdir return a list containing all the files in the given path
print(f" NO Diabetic Retia: there are { len(os.listdir(s_path_0_NODR))} images s_path_0_NODR . ")
print(f" Diabetic Retiana Mild : there are { len(os.listdir(s_path_1_Mild))} images s_path_1_Mild . ")
print(f" Diabetic Retiana Moderate : there are { len(os.listdir(s_path_2_Moderate))} images s_path_2_Moderate  . ")
print(f" Diabetic Retiana Severe : there are { len(os.listdir(s_path_3_Severe))} images s_path_3_Severe . ")
print(f" Diabetic Retiana Proliferative : there are { len(os.listdir(s_path_4_Proliferative))} images s_path_4_Proliferative . ")

 NO Diabetic Retia: there are 25810 images s_path_0_NODR . 
 Diabetic Retiana Mild : there are 2443 images s_path_1_Mild . 
 Diabetic Retiana Moderate : there are 5292 images s_path_2_Moderate  . 
 Diabetic Retiana Severe : there are 873 images s_path_3_Severe . 
 Diabetic Retiana Proliferative : there are 708 images s_path_4_Proliferative . 


# Split images into sane and desease
In this first work, I just separete between **sane** retina and retina with **desese**.
For our classification problem I will  need to structure a directory for our Retina images,  subdirectories for training and validation. These in turn will need subdirectories for 'sane' and 'desease'. 

In [ ]:
import shutil
root_dir= "./retina_CNN"

#Empty ii to prevent FieExistError if the function runs several times
if(os.path.exists(root_dir)):
  shutil.rmtree(root_dir)

def create_train_validation_dirs(root_path):
  """
  Creates directories for the train and test sets
  
  Args:
    root_path (string) - the base directory path to create subdirectories from
  
  Returns:
    None
  """  
  #define train sets
  train_dir = os.path.join(root_dir,"training")
  train_sane = os.path.join(train_dir,"sane")
  train_diabetic = os.path.join(train_dir,"diabetic")
  #define validation sets
  validation_dir = os.path.join(root_dir,"validation")
  validation_sane = os.path.join(validation_dir,"sane")
  validation_diabetic = os.path.join(validation_dir,"diabetic")
  list_dirs =[train_dir,train_sane,train_diabetic,validation_dir,validation_sane,validation_diabetic]
  print(list_dirs)

  for d in list_dirs:
    if(os.path.exists(d)):
      print(f"Path {d} already exist")
    else:
      os.makedirs(d)
      print(f"Path {d} created.")

try:
  create_train_validation_dirs(root_dir)
except FileExistsError:
  print("You should not be seeing this since the upper directory is removed beforehand")

['./retina_CNN/training', './retina_CNN/training/sane', './retina_CNN/training/diabetic', './retina_CNN/validation', './retina_CNN/validation/sane', './retina_CNN/validation/diabetic']
Path ./retina_CNN/training created.
Path ./retina_CNN/training/sane created.
Path ./retina_CNN/training/diabetic created.
Path ./retina_CNN/validation created.
Path ./retina_CNN/validation/sane created.
Path ./retina_CNN/validation/diabetic created.


Now just code the split_data function which takes in the following arguments:

* SOURCE_DIR: directory containing the files
* TRAINING_DIR: directory that a portion of the files will be copied to (will be used for training)

* VALIDATION_DIR: directory that a portion of the files will be copied to (will be used for validation)

* SPLIT_SIZE: determines the portion of images used for training.

The files will be randomized, so that the training set is a random sample of the files, and the validation set is made up of the remaining files.

For example, if SOURCE_DIR is retina/0, and SPLIT_SIZE is .9 then 90% of the images in retina/0 will be copied to the TRAINING_DIR directory and 10% of the images will be copied to the VALIDATION_DIR directory.

All images should be checked before the copy, so if they have a zero file length, they will be omitted from the copying process. 

In [ ]:
from genericpath import getsize
import random 

def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE, MAX_NUM_FILES):
  """
  Splits the data into train and test sets
  
  Args:
    SOURCE_DIR (string): directory path containing the images
    TRAINING_DIR (string): directory path to be used for training
    VALIDATION_DIR (string): directory path to be used for validation
    SPLIT_SIZE (float): proportion of the dataset to be used for training
    MAX_NUM?_FILES : limit the number of the file to be copied from source directory
    
  Returns:
    None
  """
  
  file_shuffled = []
  files_in_source_dir = os.listdir(SOURCE_DIR)
  print(f"{SOURCE_DIR} contains {len(files_in_source_dir)} files.")

  for f in files_in_source_dir:
    if(os.path.getsize(os.path.join(SOURCE_DIR,f))> 0):
      file_shuffled.append(f)
    else:
      print(f"{f} is zero lenght, skipped")

  #randomize the images and create 2 list
  random.shuffle(file_shuffled)
  n_files = len(file_shuffled)
  print(f"shuffled first step {n_files} files ")
  file_shuffled = file_shuffled[0 : min(len(files_in_source_dir),MAX_NUM_FILES)]
  n_files = len(file_shuffled)
  print(f"shuffled second step {n_files} files ")
  len_to_train = int(n_files*SPLIT_SIZE)
  train_images = file_shuffled[:len_to_train]
  validation_images = file_shuffled[len_to_train:]
  print(f"Images for training : {len(train_images) }, image for validation:  {len(validation_images)}") 

  #copy the files
  for f in train_images:
    shutil.copy(SOURCE_DIR + f , TRAINING_DIR  + f )
  print(f"files from {SOURCE_DIR} to {TRAINING_DIR} copied ")
  for f in validation_images:
    shutil.copy(SOURCE_DIR + f , VALIDATION_DIR + f)
  print(f"files from {SOURCE_DIR} to {VALIDATION_DIR} copied ")


In [ ]:
SOURCE_SANE_DIR = "/content/retina/0/"
SOURCE_DIABETIC_1 = "/content/retina/1/"
SOURCE_DIABETIC_2 = "/content/retina/2/"
SOURCE_DIABETIC_3 = "/content/retina/3/"
SOURCE_DIABETIC_4 = "/content/retina/4/"
print(f"There are {len(os.listdir(SOURCE_SANE_DIR))} images of sane")
print(f"There are {len(os.listdir(SOURCE_DIABETIC_1))} images of SOURCE_DIABETIC_1")
print(f"There are {len(os.listdir(SOURCE_DIABETIC_2))} images of SOURCE_DIABETIC_2")
print(f"There are {len(os.listdir(SOURCE_DIABETIC_3))} images of SOURCE_DIABETIC_3")
print(f"There are {len(os.listdir(SOURCE_DIABETIC_4))} images of SOURCE_DIABETIC_4")


TRAINING_DIR = "/content/retina_CNN/training/"
TRAIN_SANE = os.path.join(TRAINING_DIR,"sane/")
TRAIN_DIABETIC = os.path.join(TRAINING_DIR,"diabetic/")

VALIDATION_DIR =  "/content/retina_CNN/validation/"
VALIDATION_SANE = os.path.join(VALIDATION_DIR,"sane/")
VALIDATION_DIABETIC = os.path.join(VALIDATION_DIR,"diabetic/")


# Empty directories in case you run this cell multiple times
if len(os.listdir(TRAIN_SANE)) > 0:
  for file in os.scandir(TRAIN_SANE):
    os.remove(file.path)
if len(os.listdir(TRAIN_DIABETIC)) > 0:
  for file in os.scandir(TRAIN_DIABETIC):
    os.remove(file.path)
if len(os.listdir(VALIDATION_SANE)) > 0:
  for file in os.scandir(VALIDATION_SANE):
    os.remove(file.path)
if len(os.listdir(VALIDATION_DIABETIC)) > 0:
  for file in os.scandir(VALIDATION_DIABETIC):
    os.remove(file.path)

split_size = .8

os.getcwd()



There are 25810 images of sane
There are 2443 images of SOURCE_DIABETIC_1
There are 5292 images of SOURCE_DIABETIC_2
There are 873 images of SOURCE_DIABETIC_3
There are 708 images of SOURCE_DIABETIC_4


'/content'

In [ ]:
split_data(SOURCE_SANE_DIR, TRAIN_SANE,VALIDATION_SANE,split_size, 800)
print(f"There are {len(os.listdir(TRAIN_SANE))} images of sane for training")
print(f"There are {len(os.listdir(VALIDATION_SANE))} images of sane for validation")


/content/retina/0/ contains 25810 files.
shuffled first step 25810 files 
shuffled second step 800 files 
Images for training : 640, image for validation:  160
files from /content/retina/0/ to /content/retina_CNN/training/sane/ copied 
files from /content/retina/0/ to /content/retina_CNN/validation/sane/ copied 
There are 640 images of sane for training
There are 160 images of sane for validation


In [ ]:
split_data(SOURCE_DIABETIC_4, TRAIN_DIABETIC,VALIDATION_DIABETIC,split_size,800)
print(f"There are {len(os.listdir(TRAIN_DIABETIC))} images of diabetic 4 for training")
print(f"There are {len(os.listdir(VALIDATION_DIABETIC))} images of diabetic 4 for validation")


/content/retina/4/ contains 708 files.
shuffled first step 708 files 
shuffled second step 708 files 
Images for training : 566, image for validation:  142
files from /content/retina/4/ to /content/retina_CNN/training/diabetic/ copied 
files from /content/retina/4/ to /content/retina_CNN/validation/diabetic/ copied 
There are 566 images of diabetic 4 for training
There are 142 images of diabetic 4 for validation


In [ ]:
#split_data(SOURCE_DIABETIC_2, TRAIN_DIABETIC,VALIDATION_DIABETIC,split_size)
#print(f"There are {len(os.listdir(TRAIN_DIABETIC))} images of diabetic 2 for training")
#print(f"There are {len(os.listdir(VALIDATION_DIABETIC))} images of diabetic 2 for validation")
#split_data(SOURCE_DIABETIC_3, TRAIN_DIABETIC,VALIDATION_DIABETIC,split_size)
#print(f"There are {len(os.listdir(TRAIN_DIABETIC))} images of diabetic 3 for training")
#print(f"There are {len(os.listdir(VALIDATION_DIABETIC))} images of diabetic 3 for validation")
#split_data(SOURCE_DIABETIC_4, TRAIN_DIABETIC,VALIDATION_DIABETIC,split_size)
#print(f"There are {len(os.listdir(TRAIN_DIABETIC))} images of diabetic 4 for training")
#print(f"There are {len(os.listdir(VALIDATION_DIABETIC))} images of diabetic 4 for validation")

# Zip the files

In [ ]:
!zip -r /content/retina_severe1500_CNN.zip /content/retina_CNN

  adding: content/retina_CNN/ (stored 0%)
  adding: content/retina_CNN/training/ (stored 0%)
  adding: content/retina_CNN/training/sane/ (stored 0%)
  adding: content/retina_CNN/training/sane/17665_left.jpeg (deflated 6%)
  adding: content/retina_CNN/training/sane/24619_left.jpeg (deflated 3%)
  adding: content/retina_CNN/training/sane/20948_left.jpeg (deflated 5%)
  adding: content/retina_CNN/training/sane/43689_right.jpeg (deflated 22%)
  adding: content/retina_CNN/training/sane/19442_left.jpeg (deflated 8%)
  adding: content/retina_CNN/training/sane/38286_left.jpeg (deflated 6%)
  adding: content/retina_CNN/training/sane/14377_right.jpeg (deflated 6%)
  adding: content/retina_CNN/training/sane/17808_left.jpeg (deflated 4%)
  adding: content/retina_CNN/training/sane/19071_right.jpeg (deflated 14%)
  adding: content/retina_CNN/training/sane/10960_left.jpeg (deflated 30%)
  adding: content/retina_CNN/training/sane/13036_right.jpeg (deflated 13%)
  adding: content/retina_CNN/training/sa

In [ ]:
!cp /content/retina_severe1500_CNN.zip /content/drive/MyDrive/Colab_Notebooks/retina

In [ ]:
#test the zip file
import sys
from contextlib import closing
from zipfile import ZipFile

with closing(ZipFile("/content/retina_severe1500_CNN.zip")) as archive:
    count = len(archive.infolist())
print(count)

1515


In [ ]:
#Filegroup.py
from zipfile import ZipFile
with zipfile.ZipFile('/content/retina_severe1500_CNN.zip') as zf:
    cc = 0
    for x in zf.namelist():
      if x.endswith('/'):
        print(x)
        print(cc)
        cc=0
      else:     
       cc= cc+1
    print(cc)
    


content/retina_CNN/
0
content/retina_CNN/training/
0
content/retina_CNN/training/sane/
0
content/retina_CNN/training/diabetic/
640
content/retina_CNN/validation/
566
content/retina_CNN/validation/sane/
0
content/retina_CNN/validation/diabetic/
160
142
